# Calibration Setup Guide

### Created on Thu Dec  5 11:47:31 2019

@author: williamstanford

#### This script enables you to capture images that you can use to calibrate your camera

Things to keep in mind
    1. Plan ahead when condsidering your real world coordinate system
        - moving something from one peg to another is approximately 25.4mm 
        - with our micromanipulator, you have approximately 40 milimeters of horizontal freedom
            and 30 milimeters of vertical freedom
    2. regular increments make it easier to tell if you're off or missed a picture
        - Suggestions - 5, 8, or 10mm horizontally, 5, or 10mm vertically
    3. Save your planned real coordinates as an ods file with the X,Y,Z coordinates
        - see realPoints.ods as an example. 
        - make sure you have a space between the beginning of your cooridnates and the 
             top of the ods sheet
    3. Make sure to note the order of your cameras at this stage - you need to keep
        this consistent in later parts of processing
        
Before running, you may need to start up your cameras by running the following code in terminal/command line:

conda activate pseye
sudo chmod o+w /dev/bus/usb/001/*
sudo chmod o+w /dev/bus/usb/002/*

## Take Calibration Images

In [3]:
from pseyepy import Camera, Display
import numpy as np
import matplotlib.pyplot as plt
import os 

path = '/home/nel-lab/Documents/Behavior3D'
os.chdir(path)

Here, set num_cameras to the number of cameras you have connected.

In [4]:
num_cameras = 3
c = Camera(list(range(3)), 
           fps=[70]*num_cameras, 
           resolution=[Camera.RES_LARGE]*num_cameras, 
           colour=[False]*num_cameras)

Exception: No camera available at index 0.
Available cameras: 0

To make sure that the cameras are displaying properly, with the correct orientation, run the following code to open, then close the display.

In [ ]:
d = Display(c)

In [ ]:
d.end()

Initialize the movie. Set the number of frames to however many points you need for calibration.

In [ ]:
frames, timestamps = c.read()
frame_size = frames[0].shape
num_frames = 10
mov = np.zeros([num_frames, num_cameras, frame_size[0], frame_size[1]], dtype=np.uint8)

Set the calibration tool to your preferred start position, then run the following code. After pressing enter, adjust the calibration tool to the next position. 

You may wish to create a .csv or Excel file titled "real_coordinates.ods" with x, y, z as headers and the millimeters you plan to move the calibration tool planned out. Follow this when calibrating to avoid confusion.

In [ ]:
for i in range(num_frames): 
    frames, timestamps = c.read()
    frames, timestamps = c.read()
    mov[i] = np.array(frames)
    plt.imshow(mov[i,1,:,:])
    plt.title("Calibration point " + str(i))
    plt.pause(0.001)
    input("Press enter to view image")

Note the order of cameras by running the code below and recording which index number corresponds to which camera.

In [ ]:
for camera in range(num_cameras):
    plt.imshow(mov[i,camera,:,:])
    plt.title("Camera " + str(camera)+", Image " + str(i))    
    plt.pause(0.001)
    input("Press enter to continue")

Save movie.

In [2]:
base_name = 'recal_3_cam_'
filenames = [str(i)+'.npz' for i in range(num_cameras)]

for index,fls in enumerate(filenames):
    np.savez("mov_" + base_name + fls, movie = mov[:,index,:,:])

NameError: name 'num_cameras' is not defined

Close the cameras to end the process.

In [ ]:
c.end()

## Label Calibration Images

In [1]:
import numpy as np
import os 
import matplotlib.pyplot as plt
import pandas as pd 
from pandas_ods_reader import read_ods

Here, you will be labeling the same point from different camera views, so that the algorithm can align these images to get an idea of the 3D space. First, run the cell below to set up the function.

In [2]:
def label_images(start, end, views, obstructed_images, root_directory, video_name, realPoints):
        
    os.chdir(root_directory)
    images = end - start - len(obstructed_images)
    coords = np.zeros((images+1, 2*len(views)))
    
    for j in range(len(views)):
        path = 'images_for_' + video_name + views[j]
        os.chdir(path)
        
        p = 0
        for k in range(int(end-start)):
            i = k+start
            if (np.isin(i, obstructed_images) == False):
                image = plt.imread('fr_'+str(i).zfill(5)+'.png')
                plt.figure(1, figsize=(20,20))
                plt.imshow(image, cmap='gray')
                plt.title('image ' + str(i).zfill(5) + ' cam ' + views[j])
                s1 = 2*j
                s2 = s1+2              
                coords[p,s1:s2] = plt.ginput(1)[0] 
                p = p+1
     
        # bug where first image is double counted, so we built an array with 1
        # extra row index, here we fill this row with the pts from the final image
        i = end-1          
        image = plt.imread('fr_'+str(i).zfill(5)+'.png')
        plt.figure(1, figsize=(20,20))
        plt.imshow(image, cmap='gray')
        plt.title('image ' + str(i).zfill(5) + ' cam ' + views[j])
        s1 = 2*j
        s2 = s1+2
        coords[-1,s1:s2] = plt.ginput(1)[0] 

        i = 0
                    
        plt.close()            
        os.chdir(root_directory)
    
    coords_no_duplicates = pd.DataFrame(coords[1:,:])

    '''
    Process real points from ods or excel, this code will automatically remove
    rows before the starting image, after the ending image, and rows corresponding 
    to images obstructed
    '''
    
    real = pd.DataFrame([])
    zeros = pd.DataFrame([[0,0,0]], columns=["X", "Y", "Z"])
    rp = read_ods(realPoints,1, columns=["X", "Y", "Z"])
    # rp = pd.read_excel('realPoints.xlsx') # if pulling points from an excel file
    real = zeros.append(rp)
    real.reset_index(inplace = True, drop = True) 
    
    real = real.drop(real.index[end:])
    real = real.drop(real.index[0:start])
    
    k = 0  
    i = 0
    for i in range(real.shape[0]):
        for j in range(len(obstructed_images)):    
            if real.index[i-k] == obstructed_images[j]:          
                real = real.drop(real.index[i-k])
                k = k+1
    
    real.reset_index(inplace=True, drop=True)
    
    '''
    Add the two dataframes together
    '''
    coords_and_realPoints = pd.DataFrame([])
    coords_and_realPoints = coords_and_realPoints.append(coords_no_duplicates)
    coords_and_realPoints['X'] = real['X']
    coords_and_realPoints['Y'] = real['Y']
    coords_and_realPoints['Z'] = real['Z']
                
    return coords_and_realPoints

Below are 3 potential models.

Model 1 and 2: For 3 views. m1_end indicates the number of frames you have from one camera. realPointsPath is the path to your excel file with your list of what coordinates you photographed. For the variable m1_obstructed images, go through all of the images from each camera and note the frame number for any images where you can't see the calibration tool, as these will have to be omitted from the model. m1_views refers to the cameras you'd like to use to create hte 3D view: just list the number of the camera from the previous steps.

Models 2a and 2b have very similar variables as Model 1 and 2; however, they only accept two camera views.

Make sure you click on the same (x, y, z) position each time, or the labeling will be inaccurate.

In [ ]:
'''
for model 1
'''
root_directory = '/home/nel-lab/Documents/Behavior3D/Videos'
os.chdir(root_directory)
realPointsPath = '/home/nel-lab/Desktop/Cynthia/real_coordinates.ods'

video_name = 'mov_mov_recal_3_cam_'

m1_start = 0
m1_end = 10
m1_obstructed_images = []
m1_views = ['0','1','2'] # Cameras - front left, front right, bot

coords_m1 = label_images(m1_start, m1_end, m1_views, m1_obstructed_images, root_directory, 
                         video_name, realPointsPath)

Run the cell below to save your 3D coordinates as a csv.

In [7]:
model_1_coordinates = coords_m1
model_1_coordinates = pd.DataFrame(model_1_coordinates)
model_1_coordinates.to_csv('/home/nel-lab/Desktop/Cynthia/model_1_coordinates.csv', index=False)

NameError: name 'coords_m1' is not defined

In [ ]:
'''
for model 2 - this requires that all three cameras saw every position 
this wasn't the case for the most recent setup with 5 cameras on the wheel
you'll have to use a separate 2-camera models each of the back feet - these
models are 2a and 2b
'''
root_directory = '/home/nel-lab/Desktop/WS_tests_for_DLC2/calibration2/Calibration_Videos2'
os.chdir(root_directory)
realPointsPath = '/home/nel-lab/Desktop/WS_tests_for_DLC2/calibration2/realPoints.ods'

video_name = 'mov_recal_5_cam_'

m2_start = 15
m2_end = 70
m2_obstructed_images = [69,60,59,20,19]
m2_views = ['1','3','4'] # Cameras - back left, back right, bot

coords_m2 = label_images(m2_start, m2_end, m2_views, m2_obstructed_images, root_directory, 
                         video_name, realPointsPath)

In [ ]:
model_2_coordinates = coords_m2
model_2_coordinates = pd.DataFrame(model_2_coordinates)
model_2_coordinates.to_csv('/home/nel-lab/Desktop/WS_tests_for_DLC2/calibration2/model_2_coordinates.csv', index=False)

In [ ]:
'''
for model 2a
'''
root_directory = '/home/nel-lab/Desktop/WS_tests_for_DLC2/calibration2/Calibration_Videos2'
os.chdir(root_directory)
realPointsPath = '/home/nel-lab/Desktop/WS_tests_for_DLC2/calibration2/realPoints.ods'

video_name = 'mov_recal_5_cam_'

m2a_start = 15
m2a_end = 70
m2a_obstructed_images = [69,60,59,20,19]
m2a_views = ['1','4'] # camera's - back left and bottom

coords_m2a = label_images(m2a_start, m2a_end, m2a_views, m2a_obstructed_images, root_directory, video_name, realPointsPath)

model_2a_coordinates = coords_m2a
model_2a_coordinates = pd.DataFrame(model_2a_coordinates)
model_2a_coordinates.to_csv('/home/nel-lab/Desktop/WS_tests_for_DLC2/calibration2/model_2a_coordinates.csv', index=False)

In [ ]:
#%%
'''
for model 2b
'''
root_directory = '/home/nel-lab/Desktop/WS_tests_for_DLC2/calibration2/Calibration_Videos2'
os.chdir(root_directory)
realPointsPath = '/home/nel-lab/Desktop/WS_tests_for_DLC2/calibration2/realPoints.ods'

video_name = 'mov_recal_5_cam_'

m2b_start = 15
m2b_end = 70
m2b_obstructed_images = [69,60,59,20,19]
m2b_views = ['3','4'] # camera's - back right and bottom

coords_m2b = label_images(m2b_start, m2b_end, m2b_views, m2b_obstructed_images, root_directory, video_name, realPointsPath)

#%%
model_2b_coordinates = coords_m2b
model_2b_coordinates = pd.DataFrame(model_2b_coordinates)
model_2b_coordinates.to_csv('/home/nel-lab/Desktop/WS_tests_for_DLC2/calibration2/model_2b_coordinates.csv', index=False)

## Create a 3D structure from DeepLabCut

In [ ]:
"""
Created on Mon Dec  9 14:43:03 2019

@author: williamstanford
"""
import numpy as np
import os 
import matplotlib.pyplot as plt
import pandas as pd 
from mpl_toolkits.mplot3d import Axes3D

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict

path = '/home/nel-lab/Documents/Behavior3D/Videos/calibration_apr'

os.chdir(path)
rootDirectory = path

In [8]:
# Necessary functions
def Build_Model(coordinatePath):
    
    ''' 
    uses the location of your calibration points in the real world and fits a polynomial 
    to this data to project the pixel locations in each image to your real world coordinate system
    '''  
    
    All_Data = pd.read_csv(coordinatePath) 
    
    X_train_full, X_test, y_train_full, y_test = train_test_split( 
            All_Data.iloc[:,:-3], All_Data.iloc[:,-3:], random_state=42,  test_size = 0.25)
    
#    v = All_Data.shape[0]//2
#    X_valid, X_train = X_train_full[:v], X_train_full[v:]
#    y_valid, y_train = y_train_full[:v], y_train_full[v:]
    
    svm = SVR(kernel="poly", degree=2, C=1500, epsilon=0.01, gamma="scale")
    regr = MultiOutputRegressor(svm)
    regr.fit(X_train_full, y_train_full)
    
    test_predictions = regr.predict(X_test)
    test_MSE = mean_squared_error(test_predictions, y_test)
    print('Test MSE: ' + str(test_MSE))
    
    X = All_Data.iloc[:,:-3]
    Y = All_Data.iloc[:,-3:]
    
    CV_predictions = cross_val_predict(regr, X, Y, cv=10)
    MSE = mean_squared_error(CV_predictions, Y)
    
    print('Mean squared error: ' + str(MSE))
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    for x in range(CV_predictions.shape[0]):
        ax.scatter(CV_predictions[x,0], CV_predictions[x,1], CV_predictions[x,2], 
                   color = 'blue')
        ax.scatter(All_Data.iloc[x,-3], All_Data.iloc[x,-2], All_Data.iloc[x,-1], 
                   color = 'red')

    return regr, X, Y, CV_predictions

def PreProcess_DLC_data(df):
    
    '''
    We preprocess the CSV's output by DeepLabCut
        1. Rename each column with the bodypart and coordinate (x or y)
        2. delete redundant rows and columns 
    '''
    
    for y in range (df.shape[1]):        
        df.rename(columns={df.columns[y]:df.iloc[0,y]+'_'+df.iloc[1,y]}, inplace=True)
        
    df2 = df.iloc[2:,1:]
    
    li_columns = [col for col in df2.columns if 'likelihood' in col]
    df2 = df2.drop(columns=li_columns)
        
    return df2

'''
need to make sure that columns in each dataframe align correctly. These two
functions enable us to compare columns in two dataframes and delete the extra 
'''
    
def Diff(li1, li2):
        return (list(set(li1) - set(li2)))
    
def Standardize_columns(df1,df2):
    cols1 = df1.columns.tolist()
    cols2 = df2.columns.tolist()
    
    col_diff = Diff(cols1,cols2)
    
    for i in range(len(col_diff)):
        if col_diff[i] in df1.columns:
            df1 = df1.drop(columns=col_diff[i])
        if col_diff[i] in df2.columns:
            df2 = df2.drop(columns=col_diff[i])
            
    return df1,df2

def Predict_Real_Coordinates(df1, df2, df3, bodyparts, numCameras, regr):
    print(df1.shape, df2.shape, df3.shape)
    
    # make sure all dataframes share the same columns and delete extras
    df1,df2 = Standardize_columns(df1,df2)
    df2,df3 = Standardize_columns(df2,df3)
    
    # reorder columns to follow order of the first dataframe
    df2 = df2[df1.columns]
    df3 = df3[df1.columns]
    
    print(df1.shape, df2.shape, df3.shape)

    '''
    Use model and data generated by DeepLabCut to predict the 3D locations of every
    bodypart of interest
    '''  
    rows = df1.shape[0]
    columns = 2*numCameras
    bparts = bodyparts
    
    test_data = np.zeros((rows, columns*bparts))
    print(test_data.shape, df1.shape)
    
    for r in range(bparts):
        test_data[:,6*r] = df1.iloc[:,2*r]
        test_data[:,6*r+1] = df1.iloc[:,2*r+1]
        
        test_data[:,6*r+2] = df2.iloc[:,2*r]
        test_data[:,6*r+3] = df2.iloc[:,2*r+1]
        
        test_data[:,6*r+4] = df3.iloc[:,2*r]
        test_data[:,6*r+5] = df3.iloc[:,2*r+1]
    
    Predictions = np.zeros((rows,3*bparts))
    
    for i in range(bparts):
        Predictions[:,3*i:(3*i+3)] = regr.predict(test_data[:,6*i:(6*i+6)])
        
    return Predictions

def Predict_Real_Coordinates_2cam(df1, df2, bodyparts, numCameras, regr):
    
    # make sure all dataframes share the same columns and delete extras
    df1,df2 = Standardize_columns(df1,df2)
    print(df1.shape, df2.shape)
    
    # reorder columns to follow order of the first dataframe
    df2 = df2[df1.columns]
    
    '''
    Use model and data generated by DeepLabCut to predict the 3D locations of every
    bodypart of interest
    '''
        
    rows = df1.shape[0]
    columns = 2*numCameras
    bparts = bodyparts
    
    test_data = np.zeros((rows, columns*bparts))
    
    for r in range(bparts):
        test_data[:,4*r] = df1.iloc[:,2*r]
        test_data[:,4*r+1] = df1.iloc[:,2*r+1]
        
        test_data[:,4*r+2] = df2.iloc[:,2*r]
        test_data[:,4*r+3] = df2.iloc[:,2*r+1]

    Predictions = np.zeros((rows,3*bparts))
    
    for i in range(bparts):
        Predictions[:,3*i:(3*i+3)] = regr.predict(test_data[:,4*i:(4*i+4)])
        
    return Predictions

def Plot_Bodyparts(Predictions, bodyparts):
    
    bparts = bodyparts    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    for y in range(300):
    # for y in range(Predictions.shape[0]): # to plot full video
        plt.cla()
        for x in range(bparts):
            i = 3*x
            j = 3*x+1
            k = 3*x+2
            z = 2900
            ax.scatter(Predictions[z+y,i], Predictions[z+y,j], 
                       Predictions[z+y,k], color = 'blue')
#            ax.set_xlim([0,40])
#            ax.set_ylim([0,60])
#            ax.set_zlim([-20,15])
            ax.set_xlim([-130,-60])
            ax.set_ylim([-1,1])
            ax.set_zlim([-5,20])
            
        plt.pause(0.01)
        
    return 

Pre-process the DeepLabCut CSV for formatting.

In [ ]:
front_left = pd.read_csv('csv files/front_left_0411.csv') # The name here has been edited - these files are from the output of DLC after analyzing a video                  
front_right = pd.read_csv('csv files/front_right_0411.csv') 
bot = pd.read_csv('csv files/bot_0411.csv')

mod_front_left = PreProcess_DLC_data(front_left)
mod_front_right = PreProcess_DLC_data(front_right)
mod_bot = PreProcess_DLC_data(bot)

mod_front_left.to_csv('mod_front_left.csv', index=False)
mod_front_right.to_csv('mod_front_right.csv', index=False)
mod_bot.to_csv('mod_bot.csv', index=False)

Run the cell below which corresponds to your desired model to create the predictions based on your DeepLabCut data. Make sure you look at the pandas dataframe created to check whether or not your column labels match up between the dataframes from each camera view. Change num_cameras and bodyparts to the correct values.

In [ ]:
# MODEL 1:
coordinatePath_1 = '/home/nel-lab/Documents/Behavior3D/Videos/model_1_coordinates.csv'
#coordinatePath_1 = '/Applications/NoniCloud Desktop/GiovannucciLab/cal_images_2/model_1_coordinates.csv'
m1_regr, X, Y, cv_predictions = Build_Model(coordinatePath_1)

# Predicts real coordinates
numCameras = 3
bodyparts = 5

mod_bot = pd.read_csv('mod_bot.csv')
mod_bot = mod_bot.drop(mod_bot.columns[20:], axis=1)
mod_front_right = pd.read_csv('mod_front_right.csv') 
mod_front_left = pd.read_csv('mod_front_left.csv') 
        
m1_predictions = Predict_Real_Coordinates(mod_front_left, mod_front_right, 
                                          mod_bot, bodyparts, numCameras, m1_regr)

# saves coordinates
m1_Predictions = pd.DataFrame(m1_predictions)
m1_Predictions.to_csv('m1_Predictions.csv', index=False)

# plots bodyparts (300 frames, you can change this in the Plot_Bodyparts function)
bodyparts = 5
m1_Predictions = pd.read_csv('m1_Predictions.csv') 
m1_Predictions = m1_Predictions.to_numpy() 
Plot_Bodyparts(m1_Predictions, bodyparts)

In [ ]:
# MODEL 2
coordinatePath_2 = '/home/nel-lab/Desktop/WS_tests_for_DLC2/calibration2/model_2_coordinates.csv'
m2_regr, X, Y, cv_predictions = Build_Model(coordinatePath_2)

numCameras = 3
bodyparts = 10

mod_bot = pd.read_csv('mod_bot.csv') 
mod_back_left = pd.read_csv('mod_back_left.csv') 
mod_back_right = pd.read_csv('mod_back_right.csv') 

m2_predictions = Predict_Real_Coordinates(mod_back_left, mod_back_right, mod_bot,  
                                          bodyparts, numCameras, m2_regr)

m2_Predictions = pd.DataFrame(m2_predictions)
m2_Predictions.to_csv('m2_Predictions.csv', index=False)


bodyparts = 10
m2_Predictions = pd.read_csv('m2_Predictions.csv') 
m2_Predictions = m2_Predictions.to_numpy() 
Plot_Bodyparts(m1_Predictions, bodyparts)

In [ ]:
#Model 2A
coordinatePath_2a = '/home/nel-lab/Documents/Behavior3D/Videos/calibration_apr/model_2a_coordinates.csv'
m2a_regr, X, Y, cv_predictions = Build_Model(coordinatePath_2a)


numCameras = 2
bodyparts = 5

mod_bot = pd.read_csv('mod_bot.csv')
mod_bot_l = mod_bot.drop(mod_bot.columns[20:], axis=1)
mod_front_left = pd.read_csv('mod_front_left.csv') 

m2a_predictions = Predict_Real_Coordinates_2cam(mod_front_left, mod_bot_l, bodyparts, 
                                                numCameras, m2a_regr)

m2a_Predictions = pd.DataFrame(m2a_predictions)
m2a_Predictions.to_csv('m2a_Predictions.csv', index=False)


bodyparts = 5
m2a_Predictions = pd.read_csv('m2a_Predictions.csv') 
m2a_Predictions = m2a_Predictions.to_numpy() 
Plot_Bodyparts(m2a_Predictions, bodyparts)


In [ ]:
#MODEL 2b
coordinatePath_2b = '/home/nel-lab/Documents/Behavior3D/Videos/calibration_apr/model_2b_coordinates.csv'
m2b_regr, X, Y, cv_predictions = Build_Model(coordinatePath_2b)

numCameras = 2
bodyparts = 5

mod_bot = pd.read_csv('mod_bot.csv') 
mod_bot_r = mod_bot.drop(mod_bot.columns[10:], axis=1)
mod_front_right = pd.read_csv('mod_front_right.csv') 

m2b_predictions = Predict_Real_Coordinates_2cam(mod_front_right, mod_bot_r, bodyparts, 
                                                numCameras, m2b_regr)

m2b_Predictions = pd.DataFrame(m2b_predictions)
m2b_Predictions.to_csv('m2b_Predictions.csv', index=False)

bodyparts = 5
m2b_Predictions = pd.read_csv('m2b_Predictions.csv') 
m2b_Predictions = m2b_Predictions.to_numpy() 
Plot_Bodyparts(m2b_Predictions, bodyparts)

Now you've created the predictions (mn_Predictions.csv)! You can plot them or process them with data analysis, which can be found in William's 5th script, included in this repository.